In [ ]:
%pip install transformers accelerate pdf2image
%pip install --upgrade huggingface_hub

In [ ]:
!sudo apt-get update
!sudo apt-get install -y poppler-utils

In [ ]:
from huggingface_hub import login
login(token="")

In [ ]:
from pdf2image import convert_from_path
pdf_path = "/home/Redacted.pdf"

images = convert_from_path(pdf_path)

In [ ]:

import torch
from PIL import Image
from transformers import MllamaForConditionalGeneration, AutoProcessor

model_id = "meta-llama/Llama-3.2-11B-Vision-Instruct"

model = MllamaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
processor = AutoProcessor.from_pretrained(model_id)

In [ ]:
from datetime import datetime
from zoneinfo import ZoneInfo
mark_down = ""

def generate_mark_down(image):
    response = ""
    messages = [
        {"role": "user", "content": [
            {"type": "image"},
            {"type": "text", "text": "Extract all the data from the given image in markdown template"}
        ]}
        ]
    response += f"\n## Page {page_number}\n"
    
    input_text = processor.apply_chat_template(messages, add_generation_prompt=True)
    inputs = processor(
        image,
        input_text,
        add_special_tokens=False,
        return_tensors="pt"
    ).to(model.device)

    output = model.generate(**inputs, max_new_tokens=1000)
    response += processor.decode(output[0])
    response += "\n\n---\n"
    return response
    

for idx, image in enumerate(images):
    page_number = idx + 1
    print(f"processing page {page_number} - start time {datetime.now(ZoneInfo('Asia/Kolkata'))}")
    mark_down += generate_mark_down(image)
    print(f"processing page {page_number} - end time {datetime.now(ZoneInfo('Asia/Kolkata'))}")

    print(mark_down)

print(mark_down)

    





print(mark_down)